# scCODEX preview - Dataframe preprocessing

## Set file paths

In [ ]:
data_dir = "/Users/yuqitan/Nolan Lab Dropbox/Yuqi Tan/Collaborations/Hendrik/processed_data/14_05_23_Breast_70066_risk/Scan1/CVcol_2023_01_09_DAPI_3px/fcs/"
output_dir = "/Users/yuqitan/Nolan Lab Dropbox/Yuqi Tan/Collaborations/Hendrik/processed_data/14_05_23_Breast_70066_risk/"

if not os.path.exists(output_dir): # check if output path exist - if not generate the path
    os.makedirs(output_dir)

## Load data

In [ ]:
# this is loading the segmentation output 
df = pp_read_data(path=data_dir + 'compensated/',\
               reg_list=[], nuc_1 = 1) #this is only useful when there are multiple regions. it also helps to format the heading
df.shape # this tells you the dimension of your data

## Noramlize data

In [ ]:
# This is to normalize the data 
dfz = pp_format(data=df, 
                list_out=['first_index', 'cell_id','tile_num','z', 'x_tile','y_tile', 'size','DAPI'],
                list_keep = ['region','x','y','region_num',],
                method = "zscore")

In [ ]:
#examine unique region per tissue to decide whether or not we need x y correction
pl_catplot(dfz,X = 'x',Y='y',
           exp = 'region_num', hue = 'region_num',invert_y=True,size = 1,figsize=8)

## Noise removal

In [ ]:
dfz.columns

In [ ]:
# get the column index for the last antibody 
dfz.columns.get_loc('ICOS')

In [ ]:
# This function helps to figure out what the cut-off should be
# This is to remove top 1 % of all cells that are highly expressive for all antibodies
pl_zcount_thres(dfz = dfz, 
                col_num = 41, # last antibody index
                cut_off=0.01, #top 1% of cells
                count_bin=50) 

In [ ]:
df_nn,cc = pp_remove_noise(df=dfz, 
                           col_num=41, # this is the column index that has the last protein feature
                           z_sum_thres=47, # number obtained from the function above
                           z_count_thres=34 # number obtained from the function above
                          )

### Save denoised dataframe 

In [ ]:
df_nn.to_csv(output_dir + "df_nn_NBT_230223_NBT_70129.csv")